<h1> CAPSTONE. DISTRICTS SEGMENTATION  OF THE CITY OF HUESCA, SPAIN FOR ENTREPRENEURSHIP IN THE RESTAURANT SECTOR

<h3>1. Import all python libraries neccesary to run the Lab 

In [1]:
import pandas as pd #Pandas library
import folium # map rendering library
import requests # library to handle requests
import numpy as np
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# import k-means from clustering stage
from sklearn.cluster import KMeans

<h3>2. Load Neighborghood data

The Neighborghood data is loaded from a CSV  file ("<em>Barrios_Principales_Huesca.csv</em>") downloaded from the Huesca council website using Pandas. 

In [2]:
Huesca_data = pd.read_csv('Barrios_Principales_Huesca.csv')

<h3>3. Neighborhoods Geolocation

Geolocation data is obtained using the geocoder Nominatium from geopy. In order to obtain the require information we create a new DataFrame called <em>geopy_Data</em> where we are going to append the retreaved data from nominatium for each neighborghood in ower <em>Huesca_data</em> DataFarme. 

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>Huesca</em>, as shown below.

In [3]:
geolocator = Nominatim(user_agent="Huesca", timeout=10)

column_names = ['neighborhood', 'Direccion', 'Latitud', 'Longitud'] 

# instantiate the dataframe
geopy_Data = pd.DataFrame(columns=column_names)

for Barrio, Poblacion, Provincia in zip(Huesca_data['Barrio'], Huesca_data['Poblacion'], Huesca_data['Provincia']):
    location = geolocator.geocode('{}, {}, {}'.format(Barrio, Poblacion, Provincia))
    geopy_Data = geopy_Data.append({'neighborhood' : Barrio, 'Direccion' : location.address, 'Latitud' : location.latitude, 'Longitud' : location.longitude}, ignore_index=True)


Here I show the obtained DataFrame including Address, Latitude and Longitude for each neighborhood in Huesca 

In [4]:
geopy_Data

,neighborhood,Direccion,Latitud,Longitud
0,Casco Antiguo,"Casco Antiguo, Huesca, Hoya de Huesca / Plana ...",42.139658,-0.408978
1,La Encarnacion,"La Encarnación, Huesca, Hoya de Huesca / Plana...",42.132218,-0.417095
2,Los Olivos,"Los Olivos, Huesca, Hoya de Huesca / Plana de ...",42.128421,-0.405701
3,Maria Auxiliadora,"María Auxiliadora, Huesca, Hoya de Huesca / Pl...",42.143782,-0.415775
4,Perpetuo Socorro,"Perpetuo Socorro, Huesca, Hoya de Huesca / Pla...",42.141240,-0.398349
5,San José,"San José, Huesca, Hoya de Huesca / Plana de Ue...",42.135866,-0.414268
6,San Lorenzo,"San Lorenzo, Huesca, Hoya de Huesca / Plana de...",42.133777,-0.408018
7,Santiago,"Santiago, Huesca, Hoya de Huesca / Plana de Ue...",42.139696,-0.418861
8,Santo Domingo y San Martin,"Santo Domingo y San Martín, Huesca, Hoya de Hu...",42.137576,-0.401861


Next step is obtain Huesca geolocation using Nominatium geocoder.

Again, in order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>Huesca_explorer</em>, as shown below.

In [5]:
address = 'Huesca, Huesca'

geolocator = Nominatim(user_agent="Huesca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Huesca are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Huesca are 42.1382539, -0.4080901.


Now with Huesca and each of its neighborhood geolocation we create a map using Folium. Where the parameters used are:   
                                                                                                                                         
radius = 5                                                                                                                                         
Color = blue

and geopy_Data DataFrame

In [6]:
# create map of Huesca using latitude and longitude values
map_huesca = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, borough, in zip(geopy_Data['Latitud'], geopy_Data['Longitud'], geopy_Data['neighborhood']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_huesca)  
    
map_huesca

<h3>4. FOURSQUARE API

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
Firstable, we are goin to define the credentials and version in Foursquare.

In [7]:
CLIENT_ID = 'SXSHNA3IACLH1TGK1JDOZY5RZICEGDXP1CR2VSFPDEEXJCZ2' # your Foursquare ID
CLIENT_SECRET = 'RT1JJIY4UJP5DOZMBGATRC5TZ4113TUNA00PSQWQWNIBDLOQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SXSHNA3IACLH1TGK1JDOZY5RZICEGDXP1CR2VSFPDEEXJCZ2
CLIENT_SECRET:RT1JJIY4UJP5DOZMBGATRC5TZ4113TUNA00PSQWQWNIBDLOQ


To check the workflow, we are going to explore the first neighborhood in our DataFrame, in ower case is "<em>Casco Antiguo</em>"

In [8]:
geopy_Data.loc[0, 'neighborhood']

'Casco Antiguo'

In [9]:
neighborhood_latitude = geopy_Data.loc[0, 'Latitud'] # neighborhood latitude value
neighborhood_longitude = geopy_Data.loc[0, 'Longitud'] # neighborhood longitude value

neighborhood_name = geopy_Data.loc[0, 'neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Casco Antiguo are 42.1396579, -0.4089784.


Now, let's get the top 100 venues that are in <em>Casco Antiguo</em> within a radius of 500 meters.

In [10]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=SXSHNA3IACLH1TGK1JDOZY5RZICEGDXP1CR2VSFPDEEXJCZ2&client_secret=RT1JJIY4UJP5DOZMBGATRC5TZ4113TUNA00PSQWQWNIBDLOQ&v=20180605&ll=42.1396579,-0.4089784&radius=500&limit=100'

Send the GET request and examine the results

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e1290650f5968001b9a2cb0'},
 'response': {'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 42,
  'suggestedBounds': {'ne': {'lat': 42.144157904500005,
    'lng': -0.402921040473144},
   'sw': {'lat': 42.1351578955, 'lng': -0.41503575952685606}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '500d68b7e4b07957c3f1e916',
       'name': 'Tatau',
       'location': {'address': 'Calle Azara sn',
        'lat': 42.13848310848407,
        'lng': -0.40854334831237793,
        'labeledLatLngs': [{'label': 'display',
          'lat': 42.13848310848407,
          'lng': -0.40854334831237793}],
        'distance': 135,
        'postalCode': '22002',
     

All the information is in the items key. Before we proceed we need to define information of interest and filter dataframe, keep only columns that include venue name, and anything that is associated with location

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Tatau,Tapas Restaurant,42.138483,-0.408543
1,La Duquesa,Tapas Restaurant,42.137413,-0.408225
2,Hotel Sancho Abarca ****,Hotel,42.139663,-0.410494
3,Alt Berlin,Brewery,42.138156,-0.408021
4,Flor De Huesca,Spanish Restaurant,42.136812,-0.408304


In [14]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

42 venues were returned by Foursquare.


Now we create a function to repeat the same process to all the neighborhoods in Huesca

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called *Huesca_venues*.

In [16]:
# type your answer here

Huesca_venues = getNearbyVenues(names=geopy_Data['neighborhood'],
                                   latitudes=geopy_Data['Latitud'],
                                   longitudes=geopy_Data['Longitud']
                                  )


Casco Antiguo
La Encarnacion
Los Olivos
Maria Auxiliadora
Perpetuo Socorro
San José
San Lorenzo
Santiago
Santo Domingo y San Martin


In [17]:
print(Huesca_venues.shape)
Huesca_venues.head()

(166, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Casco Antiguo,42.139658,-0.408978,Tatau,42.138483,-0.408543,Tapas Restaurant
1,Casco Antiguo,42.139658,-0.408978,La Duquesa,42.137413,-0.408225,Tapas Restaurant
2,Casco Antiguo,42.139658,-0.408978,Hotel Sancho Abarca ****,42.139663,-0.410494,Hotel
3,Casco Antiguo,42.139658,-0.408978,Alt Berlin,42.138156,-0.408021,Brewery
4,Casco Antiguo,42.139658,-0.408978,Flor De Huesca,42.136812,-0.408304,Spanish Restaurant


In this moment we can check how many venues were returned for each neighborhood in *Huesca_venues*, using groupby 

In [18]:
Huesca_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Casco Antiguo,42,42,42,42,42,42
La Encarnacion,5,5,5,5,5,5
Los Olivos,7,7,7,7,7,7
Maria Auxiliadora,9,9,9,9,9,9
Perpetuo Socorro,5,5,5,5,5,5
San José,19,19,19,19,19,19
San Lorenzo,42,42,42,42,42,42
Santiago,8,8,8,8,8,8
Santo Domingo y San Martin,29,29,29,29,29,29


Note that the neigborhoods with more venues are *Casco Antiguo*, *San José*, *San Lorenzo* and *Santo Domingo y San Martin*.

Is important to note that there are XX unique venues in Huesca.

In [19]:
print('There are {} uniques categories.'.format(len(Huesca_venues['Venue Category'].unique())))

There are 35 uniques categories.


Now we are going to performe an One hot encoding in order to analyze Huesca's neigborhoods

In [20]:
# one hot encoding
Huesca_onehot = pd.get_dummies(Huesca_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Huesca_onehot['Neighborhood'] = Huesca_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Huesca_onehot.columns[-1]] + list(Huesca_onehot.columns[:-1])
Huesca_onehot = Huesca_onehot[fixed_columns]

Huesca_onehot.head()

,Neighborhood,Asian Restaurant,Bakery,Bar,Brewery,Burger Joint,Café,Church,Coffee Shop,Cultural Center,...,Plaza,Pub,Restaurant,Sandwich Place,Spanish Restaurant,Steakhouse,Supermarket,Tapas Restaurant,Theater,Toy / Game Store
0,Casco Antiguo,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,Casco Antiguo,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,Casco Antiguo,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Casco Antiguo,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Casco Antiguo,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [21]:
Huesca_onehot.shape

(166, 36)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [22]:
Huesca_grouped = Huesca_onehot.groupby('Neighborhood').mean().reset_index()

In [23]:
Huesca_grouped

,Neighborhood,Asian Restaurant,Bakery,Bar,Brewery,Burger Joint,Café,Church,Coffee Shop,Cultural Center,...,Plaza,Pub,Restaurant,Sandwich Place,Spanish Restaurant,Steakhouse,Supermarket,Tapas Restaurant,Theater,Toy / Game Store
0,Casco Antiguo,0.000000,0.000000,0.095238,0.02381,0.023810,0.119048,0.02381,0.000000,0.023810,...,0.047619,0.047619,0.095238,0.000000,0.095238,0.0,0.000000,0.190476,0.02381,0.000000
1,La Encarnacion,0.000000,0.000000,0.000000,0.00000,0.000000,0.200000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.400000,0.0,0.000000,0.000000,0.00000,0.000000
2,Los Olivos,0.000000,0.142857,0.000000,0.00000,0.000000,0.428571,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.142857,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000
3,Maria Auxiliadora,0.000000,0.000000,0.111111,0.00000,0.000000,0.333333,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.111111,0.000000,0.111111,0.0,0.111111,0.000000,0.00000,0.000000
4,Perpetuo Socorro,0.000000,0.000000,0.000000,0.00000,0.000000,0.400000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.00000,0.000000
5,San José,0.052632,0.052632,0.157895,0.00000,0.000000,0.210526,0.00000,0.052632,0.000000,...,0.052632,0.052632,0.052632,0.052632,0.105263,0.0,0.000000,0.052632,0.00000,0.000000
6,San Lorenzo,0.000000,0.000000,0.095238,0.02381,0.023810,0.119048,0.00000,0.000000,0.000000,...,0.071429,0.047619,0.119048,0.000000,0.095238,0.0,0.000000,0.166667,0.00000,0.000000
7,Santiago,0.000000,0.125000,0.125000,0.00000,0.000000,0.375000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.125000,0.0,0.000000,0.000000,0.00000,0.000000
8,Santo Domingo y San Martin,0.000000,0.000000,0.034483,0.00000,0.034483,0.103448,0.00000,0.000000,0.034483,...,0.034483,0.034483,0.103448,0.000000,0.103448,0.0,0.034483,0.172414,0.00000,0.034483


In [24]:
Huesca_grouped.shape

(9, 36)

In order to define what kind of restaurants are in each neigborhood we can display the top 5 venues for each one:

In [25]:
num_top_venues = 5

for hood in Huesca_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Huesca_grouped[Huesca_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Casco Antiguo----
                venue  freq
0    Tapas Restaurant  0.19
1                Café  0.12
2          Restaurant  0.10
3                 Bar  0.10
4  Spanish Restaurant  0.10


----La Encarnacion----
                venue  freq
0  Spanish Restaurant   0.4
1            Mountain   0.2
2                Café   0.2
3  Italian Restaurant   0.2
4               Plaza   0.0


----Los Olivos----
           venue  freq
0           Café  0.43
1     Restaurant  0.14
2         Bakery  0.14
3    Music Venue  0.14
4  Grocery Store  0.14


----Maria Auxiliadora----
                venue  freq
0                Café  0.33
1                 Bar  0.11
2               Hotel  0.11
3         Supermarket  0.11
4  Spanish Restaurant  0.11


----Perpetuo Socorro----
                venue  freq
0                Café   0.4
1  Falafel Restaurant   0.2
2          Steakhouse   0.2
3       Grocery Store   0.2
4                 Pub   0.0


----San José----
                venue  freq
0                Caf

Note that in the neigborhoods with more vanues ( San Lorenzo, Casco Antiguo and San Pedro) the most frequent kind of venue is Tapas Restaurant, and bars (Pubs).

The next block is a function to return the most common venues

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Huesca_grouped['Neighborhood']

for ind in np.arange(Huesca_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Huesca_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(200)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Casco Antiguo,Tapas Restaurant,Café,Bar,Spanish Restaurant,Restaurant,Plaza,Pub,Hotel,Pizza Place,Burger Joint
1,La Encarnacion,Spanish Restaurant,Italian Restaurant,Mountain,Café,Toy / Game Store,Falafel Restaurant,French Restaurant,Food & Drink Shop,Food,Fast Food Restaurant
2,Los Olivos,Café,Grocery Store,Bakery,Restaurant,Music Venue,Falafel Restaurant,French Restaurant,Food & Drink Shop,Food,Fast Food Restaurant
3,Maria Auxiliadora,Café,Spanish Restaurant,French Restaurant,Restaurant,Hotel,Bar,Supermarket,Brewery,Burger Joint,Grocery Store
4,Perpetuo Socorro,Café,Grocery Store,Steakhouse,Falafel Restaurant,Toy / Game Store,Cultural Center,French Restaurant,Food & Drink Shop,Food,Fast Food Restaurant
5,San José,Café,Bar,Spanish Restaurant,Asian Restaurant,Tapas Restaurant,Bakery,Sandwich Place,Restaurant,Pub,Plaza
6,San Lorenzo,Tapas Restaurant,Café,Restaurant,Bar,Spanish Restaurant,Plaza,Pub,Pizza Place,Ice Cream Shop,French Restaurant
7,Santiago,Café,Hotel,Bakery,Bar,Spanish Restaurant,Movie Theater,Fast Food Restaurant,Grocery Store,French Restaurant,Food & Drink Shop
8,Santo Domingo y San Martin,Tapas Restaurant,Spanish Restaurant,Café,Restaurant,Falafel Restaurant,Pizza Place,Toy / Game Store,French Restaurant,Bar,Burger Joint


<h3>5. Cluster Neighborhoods

Due that Huesca only have 9 neighborhoods, I decided to define the number of cluster in 5.

The Machine Learning algorithm to use in this analysis is K-means

In [28]:
# set number of clusters
kclusters = 5

Huesca_grouped_clustering = Huesca_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Huesca_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:200] 

array([0, 1, 4, 2, 3, 2, 0, 2, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

Huesca_merged = geopy_Data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Huesca_merged = Huesca_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='neighborhood')

Huesca_merged = Huesca_merged.dropna()

Huesca_merged = Huesca_merged.astype({"Cluster_Labels": int})

#Toronto_merged = Toronto_merged.Cluster_Labels.astype(int)

In [30]:
Huesca_merged.head(100) # check the last columns!

,neighborhood,Direccion,Latitud,Longitud,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Casco Antiguo,"Casco Antiguo, Huesca, Hoya de Huesca / Plana ...",42.139658,-0.408978,0,Tapas Restaurant,Café,Bar,Spanish Restaurant,Restaurant,Plaza,Pub,Hotel,Pizza Place,Burger Joint
1,La Encarnacion,"La Encarnación, Huesca, Hoya de Huesca / Plana...",42.132218,-0.417095,1,Spanish Restaurant,Italian Restaurant,Mountain,Café,Toy / Game Store,Falafel Restaurant,French Restaurant,Food & Drink Shop,Food,Fast Food Restaurant
2,Los Olivos,"Los Olivos, Huesca, Hoya de Huesca / Plana de ...",42.128421,-0.405701,4,Café,Grocery Store,Bakery,Restaurant,Music Venue,Falafel Restaurant,French Restaurant,Food & Drink Shop,Food,Fast Food Restaurant
3,Maria Auxiliadora,"María Auxiliadora, Huesca, Hoya de Huesca / Pl...",42.143782,-0.415775,2,Café,Spanish Restaurant,French Restaurant,Restaurant,Hotel,Bar,Supermarket,Brewery,Burger Joint,Grocery Store
4,Perpetuo Socorro,"Perpetuo Socorro, Huesca, Hoya de Huesca / Pla...",42.141240,-0.398349,3,Café,Grocery Store,Steakhouse,Falafel Restaurant,Toy / Game Store,Cultural Center,French Restaurant,Food & Drink Shop,Food,Fast Food Restaurant
5,San José,"San José, Huesca, Hoya de Huesca / Plana de Ue...",42.135866,-0.414268,2,Café,Bar,Spanish Restaurant,Asian Restaurant,Tapas Restaurant,Bakery,Sandwich Place,Restaurant,Pub,Plaza
6,San Lorenzo,"San Lorenzo, Huesca, Hoya de Huesca / Plana de...",42.133777,-0.408018,0,Tapas Restaurant,Café,Restaurant,Bar,Spanish Restaurant,Plaza,Pub,Pizza Place,Ice Cream Shop,French Restaurant
7,Santiago,"Santiago, Huesca, Hoya de Huesca / Plana de Ue...",42.139696,-0.418861,2,Café,Hotel,Bakery,Bar,Spanish Restaurant,Movie Theater,Fast Food Restaurant,Grocery Store,French Restaurant,Food & Drink Shop
8,Santo Domingo y San Martin,"Santo Domingo y San Martín, Huesca, Hoya de Hu...",42.137576,-0.401861,0,Tapas Restaurant,Spanish Restaurant,Café,Restaurant,Falafel Restaurant,Pizza Place,Toy / Game Store,French Restaurant,Bar,Burger Joint


Finally, let's visualize the resulting clusters

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=14)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Huesca_merged['Latitud'], Huesca_merged['Longitud'], Huesca_merged['neighborhood'], Huesca_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [32]:
Huesca_merged.loc[Huesca_merged['Cluster_Labels'] == 0, Huesca_merged.columns[[0] + list(range(5, Huesca_merged.shape[1]))]]

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Casco Antiguo,Tapas Restaurant,Café,Bar,Spanish Restaurant,Restaurant,Plaza,Pub,Hotel,Pizza Place,Burger Joint
6,San Lorenzo,Tapas Restaurant,Café,Restaurant,Bar,Spanish Restaurant,Plaza,Pub,Pizza Place,Ice Cream Shop,French Restaurant
8,Santo Domingo y San Martin,Tapas Restaurant,Spanish Restaurant,Café,Restaurant,Falafel Restaurant,Pizza Place,Toy / Game Store,French Restaurant,Bar,Burger Joint


In [33]:
Huesca_merged.loc[Huesca_merged['Cluster_Labels'] == 1, Huesca_merged.columns[[0] + list(range(5, Huesca_merged.shape[1]))]]

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,La Encarnacion,Spanish Restaurant,Italian Restaurant,Mountain,Café,Toy / Game Store,Falafel Restaurant,French Restaurant,Food & Drink Shop,Food,Fast Food Restaurant


In [34]:
Huesca_merged.loc[Huesca_merged['Cluster_Labels'] == 2, Huesca_merged.columns[[0] + list(range(5, Huesca_merged.shape[1]))]]

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Maria Auxiliadora,Café,Spanish Restaurant,French Restaurant,Restaurant,Hotel,Bar,Supermarket,Brewery,Burger Joint,Grocery Store
5,San José,Café,Bar,Spanish Restaurant,Asian Restaurant,Tapas Restaurant,Bakery,Sandwich Place,Restaurant,Pub,Plaza
7,Santiago,Café,Hotel,Bakery,Bar,Spanish Restaurant,Movie Theater,Fast Food Restaurant,Grocery Store,French Restaurant,Food & Drink Shop


In [35]:
Huesca_merged.loc[Huesca_merged['Cluster_Labels'] == 3, Huesca_merged.columns[[0] + list(range(5, Huesca_merged.shape[1]))]]

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Perpetuo Socorro,Café,Grocery Store,Steakhouse,Falafel Restaurant,Toy / Game Store,Cultural Center,French Restaurant,Food & Drink Shop,Food,Fast Food Restaurant


In [36]:
Huesca_merged.loc[Huesca_merged['Cluster_Labels'] == 4, Huesca_merged.columns[[0] + list(range(5, Huesca_merged.shape[1]))]]

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Los Olivos,Café,Grocery Store,Bakery,Restaurant,Music Venue,Falafel Restaurant,French Restaurant,Food & Drink Shop,Food,Fast Food Restaurant
